
#1. 파일 읽기

> Quiz
- 타이타닉 파일을 읽어 들여 상위 3개의 데이터만 출력하시오

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import pandas as pd

filepath = '/content/drive/MyDrive/4. 머신러닝/2. 데이터 분석을 위한 기초 통계 수학/3. 파이프라인/data/titanic.csv'
df_titanic = pd.read_csv(filepath)
df_titanic.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


#2. 데이터 클린징

##2.1 결측치 처리

In [ ]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


> Quiz. 다음과 같이 동작되도록 코딩하시오
- Age, Cabin, Embarked에 결측치가 발생된 것을 알 수 있다. Age는 나이로 평균을 이용하여 결측치를 해결하고 나머지 두 feature는 N이라는 값을 넣어 결측치를 해결하시오

In [4]:
round(df_titanic['Age'].mean(), 0)

30.0

In [5]:
df_titanic['Age'].fillna(round(df_titanic['Age'].mean(), 0), inplace=True)
df_titanic['Cabin'].fillna('N', inplace=True)
df_titanic['Embarked'].fillna('N', inplace=True)

# 각 feature에 대한 합
# df_titanic.isnull().sum()
# 각 feature의 총합
df_titanic.isnull().sum().sum()

0

>모든 결측치의 합이 0으로 나타난다. 결국 결측치는 해결되었다.

##2.2 이상치 처리

In [6]:
print('성별 데이터 분포')
print(df_titanic['Sex'].value_counts())
print('객실 데이터 분포')
print(df_titanic['Cabin'].value_counts())
print('항구 데이터 분포')
print(df_titanic['Embarked'].value_counts())

성별 데이터 분포
male      577
female    314
Name: Sex, dtype: int64
객실 데이터 분포
N              687
G6               4
C23 C25 C27      4
B96 B98          4
F2               3
              ... 
E10              1
B4               1
B80              1
D15              1
D19              1
Name: Cabin, Length: 148, dtype: int64
항구 데이터 분포
S    644
C    168
Q     77
N      2
Name: Embarked, dtype: int64


> 성별 확인 결과 특이점은 없었으며 객실과 항구 정보는 이전 코드에서 변경한 데이터 이외의 특이점은 없었다.

In [ ]:
df_titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


>수치 데이터에서는 이상치가 발견되지 않았다.

##2.3 문자열 수치화

> 모델 생성 시 문자열이 있으면 문제가 발생할 수 있음으로 문자열을 수치 데이터로 변환한다.

In [11]:
from sklearn import preprocessing

features = ['Cabin', 'Sex', 'Embarked']

for feature in features:
  # LabelEncode 로드
  le = preprocessing.LabelEncoder()
  # feature의 크기 설정
  le = le.fit(df_titanic[feature])
  # 데이터 변환
  df_titanic[feature] = le.transform(df_titanic[feature])

df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,146,3
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,81,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,146,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,55,3
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,146,3


> LabelEncoder는 문자를 수치 데이터로 변경해 준다. 생존률에 관한 모델을 생성하려면 문자열이 존재하는 경우 에러가 발생된다. 따라서 위와 같이 Cabin, Sex, Embarked를 수치 데이터로 변경했다. 

> 내부적으로 각 데이터를 중복을 제거한 후 정렬하고 index 정보를 반환하는 기능을 한다.

In [12]:
# 생존 확인에서 Name과Ticket은 필요가 없음으로 제거
df_titanic.drop(['Name', 'Ticket'], axis=1, inplace=True)

#3. 모델 만들기

##3.1 데이터 분리


> Quiz. 생존에 따른 데이터를 분석할 것이다. 다음 조건에 맞게 훈련용데이터와 테스트용 데이터를 추출하시오
- Label 정보는 Survived를 이용한다.
- 나머지 정보를 Data로 활용한다.
- 테스트 데이터는 30%를 사용한다.
- random_state는 62로 고정한다.

In [13]:
from sklearn.model_selection import train_test_split

# Label 정보는 Survived를 이용한다.
df_label = df_titanic['Survived']
# 나머지 정보를 Data로 활용한다.
df_data = df_titanic.drop(['Survived'], axis=1)

# 테스트 데이터는 30%를 사용한다.
# random_state는 62로 고정한다.
X_train, X_test, y_train, y_test = train_test_split(df_data, df_label, test_size=0.3, random_state=62)

##3.2 결정트리

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
# 모델 로드
dt_clf = DecisionTreeClassifier()
# 학습시키기
dt_clf.fit(X_train, y_train)
# 예측 결과 얻기
pred = dt_clf.predict(X_test)

# 정확도
print('DecisionTreeClassifier 정확도 : {0:.4f}'.format(accuracy_score(y_test, pred)))

DecisionTreeClassifier 정확도 : 0.7948


##3.3 랜텀 포레스트

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# 모델 로드
rf_clf = RandomForestClassifier()
# 학습시키기
rf_clf.fit(X_train, y_train)
# 예측 결과 얻기
pred = rf_clf.predict(X_test)

# 정확도
print('RandomForestClassifier 정확도 : {0:.4f}'.format(accuracy_score(y_test, pred)))

RandomForestClassifier 정확도 : 0.8582


##3.3 로지스틱 회귀

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# 모델 로드
lr = LogisticRegression()
# 학습시키기
lr.fit(X_train, y_train)
# 예측 결과 얻기
pred = lr.predict(X_test)

# 정확도
print('LogisticRegression 정확도 : {0:.4f}'.format(accuracy_score(y_test, pred)))

LogisticRegression 정확도 : 0.8172


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


##3.4 경고 처리

> /usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

>Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html

>Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)

> 위와 같은 경고가 나타나면 다음과 같이 해결할 수 있다.
- lr = LogisticRegression(max_iter=1000)

> max_iter 부분에 적당한 크기의 숫자를 넣으면 경고를 제거할 수 있다.

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# max_iter=1000 추가
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
pred = lr.predict(X_test)

print('LogisticRegression 정확도 : {0:.4f}'.format(accuracy_score(y_test, pred)))

LogisticRegression 정확도 : 0.8022


#4. 교차 검증

##4.1 kFold

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

def exec_kfold(clf, folds):
    # 폴드 세트가 5개인 kFold 객체 생성, 폴드 수 만큼 예측결과 저장을 위한 리스트 객체 생성
    kfold = KFold(n_splits=folds)
    scores = []
    
    # kFold 교차검증 수행
    for iter_count, (train_index, test_index) in enumerate(kfold.split(df_titanic_X)):
        # X_train_df 데이터에서 교차검증별로 학습과 검증 데이터를 가리키는 인덱스 생성
        X_train, X_test = df_titanic_X.values[train_index], df_titanic_X.values[test_index]
        y_train, y_test = df_titanic_y.values[train_index], df_titanic_y.values[test_index]
        # Classifier 학습, 예측, 정확도 계산
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        scores.append(accuracy)
        print("교차검증 {0} 정확도: {1:.4f}".format(iter_count, accuracy))
        # 5개 fold에서 평균 정확도 계산
    mean_score = np.mean(scores)
    print("평균 정확도 : {0:.4f}".format(mean_score))
    
# exec_kfold 호출
exec_kfold(dt_clf, 5)
print('===========================================')
exec_kfold(rf_clf, 5)
print('===========================================')
exec_kfold(lr_clf, 5)

교차검증 0 정확도: 0.7151
교차검증 1 정확도: 0.7809
교차검증 2 정확도: 0.7865
교차검증 3 정확도: 0.7978
교차검증 4 정확도: 0.3652
평균 정확도 : 0.6891
교차검증 0 정확도: 0.7765
교차검증 1 정확도: 0.8202
교차검증 2 정확도: 0.8427
교차검증 3 정확도: 0.7978
교차검증 4 정확도: 0.8315
평균 정확도 : 0.8137
교차검증 0 정확도: 0.7542
교차검증 1 정확도: 0.8034
교차검증 2 정확도: 0.7640
교차검증 3 정확도: 0.7360
교차검증 4 정확도: 0.8034
평균 정확도 : 0.7722


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


##4.2 cross_val_score

In [ ]:
from sklearn.model_selection import cross_val_score

def cross_score(clf):
  scores = cross_val_score(clf, df_titanic_X, df_titanic_y, cv=5)
  for iter_count, accuracy in enumerate(scores):
      print("교차검증 {0} 정확도: {1:.4f}".format(iter_count, accuracy))
    
  print("평균정확도: {0: .4f}".format(np.mean(scores)))

cross_score(dt_clf)
print('===========================================')
cross_score(rf_clf)
print('===========================================')
cross_score(lr_clf)

##4.3 GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth': [2, 3, 5, 10],
              'min_samples_split' : [2, 3, 5],
              'min_samples_leaf' : [1, 5, 8]
              }
def grid_search(clf, opt):
  grid_dclf = GridSearchCV(clf, param_grid=parameters, scoring='accuracy', cv=5)
  grid_dclf.fit(X_train, y_train)

  print("GridSearch 최적 하이퍼 파라미터: ", grid_dclf.best_params_)
  print("GridSearch 최고 정확도:{:.4f} ".format(grid_dclf.best_score_))
  best_dclf = grid_dclf.best_estimator_

  # GridSearch의 최적 하이퍼파라미터로 학습된 Estimator로 예측 및 평가 수행
  predictions = best_dclf.predict(X_test)
  accuracy = accuracy_score(y_test, predictions)
  print('테스트 세트에서의 DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy))

grid_search(dt_clf, True)
print('===========================================')
grid_search(rf_clf, True)

GridSearch 최적 하이퍼 파라미터:  {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}
GridSearch 최고 정확도:0.8005 
테스트 세트에서의 DecisionTreeClassifier 정확도: 0.8101
GridSearch 최적 하이퍼 파라미터:  {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearch 최고 정확도:0.8300 
테스트 세트에서의 DecisionTreeClassifier 정확도: 0.8101


In [ ]:
from sklearn.model_selection import GridSearchCV

parameters={
        'C': [2, 3, 5, 10],
        'penalty':['l2']
    }

def grid_search(clf, opt):    
  grid_dclf = GridSearchCV(clf, param_grid=parameters, scoring='accuracy', cv=5)
  grid_dclf.fit(X_train, y_train)

  print("GridSearch 최적 하이퍼 파라미터: ", grid_dclf.best_params_)
  print("GridSearch 최고 정확도:{:.4f} ".format(grid_dclf.best_score_))
  best_dclf = grid_dclf.best_estimator_

  # GridSearch의 최적 하이퍼파라미터로 학습된 Estimator로 예측 및 평가 수행
  predictions = best_dclf.predict(X_test)
  accuracy = accuracy_score(y_test, predictions)
  print('테스트 세트에서의 DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy))

grid_search(lr_clf, False)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


GridSearch 최적 하이퍼 파라미터:  {'C': 5, 'penalty': 'l2'}
GridSearch 최고 정확도:0.7922 
테스트 세트에서의 DecisionTreeClassifier 정확도: 0.8045


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


> 경고 해결
* Increase the number of iterations (max_iter) or scale the data as shown in: 반복횟수를 늘리거나 데이터를 확장하라고 한다. 다음과 같이 수정하자

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters={
        'C': [2, 3, 5, 10],
        'penalty':['l2'], 
        # 반복 횟수 늘리기
        'max_iter':[1000]
    }

def grid_search(clf, opt):    
  grid_dclf = GridSearchCV(clf, param_grid=parameters, scoring='accuracy', cv=5)
  grid_dclf.fit(X_train, y_train)

  print("GridSearch 최적 하이퍼 파라미터: ", grid_dclf.best_params_)
  print("GridSearch 최고 정확도:{:.4f} ".format(grid_dclf.best_score_))
  best_dclf = grid_dclf.best_estimator_

  # GridSearch의 최적 하이퍼파라미터로 학습된 Estimator로 예측 및 평가 수행
  predictions = best_dclf.predict(X_test)
  accuracy = accuracy_score(y_test, predictions)
  print('테스트 세트에서의 DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy))

grid_search(lr_clf, False)

GridSearch 최적 하이퍼 파라미터:  {'C': 2, 'max_iter': 1000, 'penalty': 'l2'}
GridSearch 최고 정확도:0.8020 
테스트 세트에서의 DecisionTreeClassifier 정확도: 0.7989


> 두 번째 경고
* Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

> l2 또는 none을 지원한다고 한다. 따라서 다음과 같이 수정하자.

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters={
        'C': [2, 3, 5, 10],
        #l1 제거
        'penalty':['l2'], 
        # 반복 횟수 늘리기
        'max_iter':[1000]
    }

def grid_search(clf, opt):    
  grid_dclf = GridSearchCV(clf, param_grid=parameters, scoring='accuracy', cv=5)
  grid_dclf.fit(X_train, y_train)

  print("GridSearch 최적 하이퍼 파라미터: ", grid_dclf.best_params_)
  print("GridSearch 최고 정확도:{:.4f} ".format(grid_dclf.best_score_))
  best_dclf = grid_dclf.best_estimator_

  # GridSearch의 최적 하이퍼파라미터로 학습된 Estimator로 예측 및 평가 수행
  predictions = best_dclf.predict(X_test)
  accuracy = accuracy_score(y_test, predictions)
  print('테스트 세트에서의 DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy))

grid_search(lr_clf, False)

GridSearch 최적 하이퍼 파라미터:  {'C': 2, 'max_iter': 1000, 'penalty': 'l2'}
GridSearch 최고 정확도:0.8020 
테스트 세트에서의 DecisionTreeClassifier 정확도: 0.7989


> 결론적으로는 랜덤 포레스트를 이용한 후 GridSearchCV 튜닝하는 것이 가장 좋은 성능을 낸다.